In [1]:
import os

In [2]:
%pwd

'/Users/suryasadasivam/DataScience /project/MLproject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/suryasadasivam/DataScience /project/MLproject'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/suryasadasivamm/End_to_end_Mlproject_modular_code.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="suryasadasivamm"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b6642fdc85f6486338f8b5daea8dce29577793d1"

In [6]:
%pwd

'/Users/suryasadasivam/DataScience /project/MLproject'

In [33]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_data_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri:str

In [34]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json


In [43]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            self.schema=read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_data_path=config.model_data_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/suryasadasivamm/End_to_end_Mlproject_modular_code.mlflow"
        )

        return model_evaluation_config


In [44]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [49]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def eval_metric(self,actual,pred):
        rsme=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rsme,mae,r2

    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_data_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)

            (rsme,mae,r2)=self.eval_metric(test_y,predicted_qualities)

            #saving metric as local
            scores={'rsme':rsme,'mae':mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('rsme',rsme)
            mlflow.log_metric('r2',r2)
            mlflow.log_metric("mae",mae)


            #Model registry does not work with file store
            if tracking_url_type_store!="file":

                mlflow.sklearn.log_model(model,"model",registered_model_name="Elasticmodel")
            else:
                mlflow.sklearn.log_model(model,"model")



In [50]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e 

[2025-05-01 13:55:56,889:INFO:common:yaml file:config/config.yamlloaded successfully]
[2025-05-01 13:55:56,891:INFO:common:yaml file:params.yamlloaded successfully]
[2025-05-01 13:55:56,892:INFO:common:yaml file:schema.yamlloaded successfully]
[2025-05-01 13:55:56,893:INFO:common:created directory at :artifacts]
[2025-05-01 13:55:56,893:INFO:common:created directory at :artifacts/model_evaluation]
[2025-05-01 13:55:57,246:INFO:common:json file saved at:artifacts/model_evaluation/metrics.json]


/Users/suryasadasivam/DataScience /project/MLproject/venv/lib/python3.11/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/suryasadasivam/DataScience /project/MLproject/venv/lib/python3.11/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/suryasadasivam/DataScience /project/MLproject/venv/lib/python3.11/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
2025/05/01 13:56:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Elasticmodel'.
2025/05/01 13:56:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creati

🏃 View run gregarious-sloth-530 at: https://dagshub.com/suryasadasivamm/End_to_end_Mlproject_modular_code.mlflow/#/experiments/0/runs/60e58e5368dc4a1dbdb2d582bad50c89
🧪 View experiment at: https://dagshub.com/suryasadasivamm/End_to_end_Mlproject_modular_code.mlflow/#/experiments/0
